
# VLSI DESIGN PROBLEM USING MIXED INTEGER LINEAR PROGRAMMING-FIXED

""" Team members : vida Zahedi (vida.zahedi@studio.unibo.it) - Samral Tahirli (samral.tahirli@studio.unibo.it) """

In [1]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 22.3 MB/s eta 0:00:00


In [53]:
#importing requiredlibraries
import time
import math
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from itertools import combinations

In [54]:
#for reading the data and making the inputs
def read_instance(file_name):
    with open(file_name, 'r') as f:
        Width = int(f.readline().strip())
        Number = int(f.readline().strip())
        Pieces = np.array([list(map(int, line.strip().split())) for line in f.readlines()])
    return Width, Number, Pieces


In [61]:
#a function to solve the problem
def vlsi_optimization(Width, Number, Pieces):
    x_ith = Pieces[:, 0]
    y_ith = Pieces[:, 1]
    min_height = sum(x_ith * y_ith) // Width
    max_height = math.ceil(sum(y_ith)/(Width//max(x_ith)))
    M1 = 10**6
    M2 = 10**6
    Solver = gp.Model('VLSI-DESIGN-FIXED')
    OptHeight = Solver.addVar(name='OptHeight', vtype=GRB.INTEGER, lb=min_height, ub=max_height)
    x = Solver.addVars(list(range(Number)), name="x", vtype=gp.GRB.INTEGER, lb=0, ub=Width - x_ith.min())
    y = Solver.addVars(list(range(Number)), name="y", vtype=gp.GRB.INTEGER, lb=0, ub=max_height - y_ith.min())
    comb = Solver.addVars([(i,j,k) for (i,j) in combinations(x,2) for k in range(4)], name='comb', vtype=gp.GRB.BINARY)
    Solver.addConstrs((x[i] + x_ith[i] <= Width for i in range(Number)), name='c1')
    Solver.addConstrs((y[i] + y_ith[i] <= OptHeight for i in range(Number)), name='c2')
    Solver.addConstrs((comb.sum(i,j,'*') >= 1 for (i,j) in combinations(x,2)), name = 'c3')
    Solver.addConstrs((x[j] + x_ith[j] <= x[i]  + M1 * (1 - comb[i,j,0]) for (i,j) in combinations(x,2)), name = 'c4')
    Solver.addConstrs((x[i] + x_ith[i] <= x[j] + M1 * (1 - comb[i,j,1]) for (i,j) in combinations(x,2)), name = 'c5')
    Solver.addConstrs((y[j] + y_ith[j] <= y[i] + M2 * (1 - comb[i,j,2]) for (i,j) in combinations(x,2)), name = 'c6')
    Solver.addConstrs((y[i] + y_ith[i] <= y[j] + M2 * (1 - comb[i,j,3]) for  (i,j) in combinations(x,2)), name = 'c7')
    Solver.setObjective(OptHeight, GRB.MINIMIZE)
    Solver.optimize()
    if Solver.status == GRB.OPTIMAL:
        minimized_height = Solver.getVarByName('OptHeight').x
        print('Minimized height:', minimized_height)
        for i in range(Number):
            print('Circuit', i, ':', 'x =', x[i].x, 'y =', y[i].x)
    else:
        print('Optimization failed')
    Solver.update()
    return  x, y,OptHeight 


In [62]:
#calling the functions to solve instances
Num= 2
start_time = time.time()
file_name = "/content/ins-"+str(Num)+".txt"
Width, Number, Pieces = read_instance(file_name)
#vlsi_optimization(width, Number, pieces)
vlsi_optimization(Width, Number, Pieces)
#print the execution time 
timing = (time.time() - start_time)
print(timing,"execution-time")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 60 rows, 51 columns and 175 nonzeros
Model fingerprint: 0x5b3332e8
Variable types: 0 continuous, 51 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 9.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 9 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+00, best bound 9.000000000000e+00, gap 0.0000%
Minimized height: 9.0
Circuit 0 : x = 6.0 y = 0.0
Circuit 1 : x = 3.0 y = 5.0
Circuit 2 : x = 3.0 y = 0.0
Circuit 3 : x = 6.0 y = 3.0
Circuit 4 : x = 0.0 y = 0.0
0.047510862